In [1]:
from pathlib import Path
from cpsat_autotune import tune_for_quality_within_timelimit, import_model

/home/krupke/anaconda3/envs/mo312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# Automatically load all models from the 'models' directory
def load_models(directory: str) -> list:
    # model_paths = Path(directory).rglob("*.pb")
    model_paths = [Path("models/model_proto_-84479006250281510.pb")]
    return [import_model(path) for path in model_paths]



In [3]:

models = load_models("models")

In [4]:


tune_for_quality_within_timelimit(models[0], 1, 0, "maximize")


[I 2024-08-13 21:53:13,804] A new study created in memory with name: no-name-78ff7e23-8905-40cc-bb22-33ed5cf8b741


Computing baseline
Baseline: [156828.0, 157145.0, 157025.0, 157002.0, 155445.0, 157002.0, 157002.0, 156756.0, 157145.0, 157145.0, 157161.0, 157060.0, 155448.0, 156890.0, 157002.0, 156756.0, 156807.0, 156852.0, 157161.0, 157002.0, 155335.0, 156626.0, 156611.0, 155559.0, 156830.0, 156922.0, 157161.0, 156873.0, 156836.0, 157002.0]


[I 2024-08-13 21:53:55,054] Trial 0 finished with value: 156840.8 and parameters: {'use_lns_only': False, 'repair_hint': False, 'use_lb_relax_lns': False, 'preferred_variable_order': 0, 'use_erwa_heuristic': False, 'linearization_level': 1, 'fp_rounding': 3, 'randomize_search': False, 'diversify_lns_params': False, 'add_objective_cut': False, 'use_objective_lb_search': False, 'use_objective_shaving_search': False, 'search_branching': 0, 'cut_level': 1, 'max_all_diff_cut_size': 1, 'symmetry_level': 2, 'max_presolve_iterations': 2, 'cp_model_probing_level': 2, 'presolve_probing_deterministic_time_limit': 4, 'presolve_bve_threshold': 1, 'ignore_subsolvers:default_lp': False, 'ignore_subsolvers:fixed': False, 'ignore_subsolvers:no_lp': False, 'ignore_subsolvers:max_lp': False, 'ignore_subsolvers:pseudo_costs': False, 'ignore_subsolvers:reduced_costs': False, 'ignore_subsolvers:quick_restart': False, 'ignore_subsolvers:quick_restart_no_lp': False, 'ignore_subsolvers:lb_tree_search': False, 

------------------------------------------------------------
Parameters:
	search_branching: 6
	cp_model_probing_level: 0
	presolve_bve_threshold: 1
	presolve_probing_deterministic_time_limit: 1
	cut_level: 0
	diversify_lns_params: True
	symmetry_level: 1
	ignore_subsolvers: ('max_lp', 'pseudo_costs', 'quick_restart_no_lp')
	max_all_diff_cut_size: 2
	preferred_variable_order: 1
	use_erwa_heuristic: True
	linearization_level: 0
	max_presolve_iterations: 0
Difference to default: 566.3000000000175
This is significant: True
------------------------------------------------------------
------------------------------------------------------------
Parameters:
	max_all_diff_cut_size: 1
	presolve_probing_deterministic_time_limit: 4
	presolve_bve_threshold: 1
	max_presolve_iterations: 2
Difference to default: 94.5
This is significant: False
------------------------------------------------------------
------------------------------------------------------------
Parameters:
	cp_model_probing_level: 

({'search_branching': 6,
  'cp_model_probing_level': 0,
  'presolve_bve_threshold': 1,
  'presolve_probing_deterministic_time_limit': 1,
  'cut_level': 0,
  'diversify_lns_params': True,
  'symmetry_level': 1,
  'ignore_subsolvers': ('max_lp', 'pseudo_costs', 'quick_restart_no_lp'),
  'max_all_diff_cut_size': 2,
  'preferred_variable_order': 1,
  'use_erwa_heuristic': True,
  'linearization_level': 0,
  'max_presolve_iterations': 0},
 566.3000000000175,
 True)